# qm_geo_opt — Run quantum accurate geometry optimisation on a small molecule

This notebook shows how to geometry optimise a small molecule using RIMP2 accurate gradients.

# 0) Complete example
See the [sample notebook](/Quickstarts/qm_geo_opt_calculation-sample.ipynb) for a complete demonstration.

# 1) Setup

## 1.0) Imports

In [ ]:
import os
from glob import glob
from pathlib import Path
import json

import rush

## 1.1) Configuration

In [ ]:
# Define our project information
EXPERIMENT = "rush qm geo opt notebook"
MOLECULE_NAME = "cyclobutane"
TAGS = ["qdx", EXPERIMENT, MOLECULE_NAME]

In [ ]:
# |hide
WORK_DIR = Path.home() / "qdx" / EXPERIMENT

if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)
os.makedirs(WORK_DIR, exist_ok=True)

## 1.2) Build your client

In [ ]:
# By using the `build_provider_with_functions` method, we will also build
# helper functions calling each module
client = await rush.build_provider_with_functions(batch_tags=TAGS)

In [ ]:
# |hide
client = await rush.build_provider_with_functions(
    batch_tags=TAGS, workspace=WORK_DIR, restore_by_default=True
)

## 1.3) Get the qm_geo_opt rush module

In [ ]:
# Get our latest modules as a dict[module_name, module_path]
# If a lock file exists, load it so that the run is reproducible
# This will be done automatically if you use the `build_provider_with_functions`
# method
modules = await client.get_latest_module_paths()

In [ ]:
for name, path in sorted(modules.items()):
    print(f"{name}: {path}")

auto3d: github:talo/tengu-auto3d/4896c5559f73046314ac4aaa0f660d86a0d259d7#auto3d_tengu
concat: github:talo/tengu-prelude/f506c7ead174cdb7e8d1725139254bb85c6b62f8#concat
convert: github:talo/tengu-prelude/f506c7ead174cdb7e8d1725139254bb85c6b62f8#convert
dubai: github:talo/Dubai/4a177b6f5711de65abf0c8856adf3c2604ca228d#dubai_tengu
fragment: github:talo/tengu-prelude/f506c7ead174cdb7e8d1725139254bb85c6b62f8#fragment
fragment_aa: github:talo/tengu-prelude/f506c7ead174cdb7e8d1725139254bb85c6b62f8#fragment_aa
gmx: github:talo/gmx_tengu_support/75f745b09ec24280298b265e18127fcd41747be7#gmx_tengu
gmx_resume: github:talo/gmx_tengu_support/75f745b09ec24280298b265e18127fcd41747be7#gmx_resume_tengu
hermes_energy: github:talo/tengu-prelude/f506c7ead174cdb7e8d1725139254bb85c6b62f8#hermes_energy
hermes_energy_batch: github:talo/tengu-prelude/f506c7ead174cdb7e8d1725139254bb85c6b62f8#hermes_energy_batch
pbsa: github:talo/pbsa-cuda/85b807d2fa6b1ea843440d93a2644ce891e41d6d#pbsa_tengu
pdb2pqr: github:talo/

In [ ]:
help(client.qm_geo_opt)

AttributeError: 'Provider' object has no attribute 'qm_geo_opt'

# 2) Run geometry optimisation

## 2.0) Create input geometry
We will be creating a QDXF Conformer from an unoptimised cyclobutane XYZ

In [ ]:
def xyz_to_qdxf_topology(xyz_data: str):
    symbols = []
    geometry = []
    for line in xyz_data.splitlines()[2:]:
        symbol, x, y, z = line.split()
        symbols.append(symbol)
        geometry.extend([float(x), float(y), float(z)])
    return {"topology": {"symbols": symbols, "geometry": geometry}}


def qdxf_topology_to_xyz(qdxf_in: dict, name: str):
    symbols = qdxf_in["topology"]["symbols"]
    geometry = qdxf_in["topology"]["geometry"]
    xyz_data = f"{len(symbols)}\n{name}\n"
    for i in range(len(symbols)):
        symbol = symbols[i]
        x, y, z = geometry[3 * i : 3 * i + 3]
        xyz_data += f"  {symbol}"
        for coord in [x, y, z]:
            xyz_data += "     "
            if coord >= 0:
                xyz_data += " "
            xyz_data += f"{(coord):.4f}"
        xyz_data += "\n"
    return xyz_data

In [ ]:
cyclobutane_xyz = """12
cyclobutane
  C     -0.750      0.452     -0.417
  C     -0.696     -0.588      0.609
  C      0.820     -0.678      0.537
  C      0.892      0.417     -0.428
  H     -1.285      1.273      0.066
  H     -1.328      0.080     -1.263
  H     -1.225     -1.507      0.366
  H     -1.029     -0.162      1.555
  H      1.158     -1.594      0.054
  H      1.310     -0.477      1.488
  H      1.432      0.009     -1.290
  H      1.506      1.236     -0.056
"""

In [ ]:
cyclobutane_qdxf = xyz_to_qdxf_topology(cyclobutane_xyz)

In [ ]:
# write the qdxf file to disk
cyclobutane_qdxf_input_file = Path(WORK_DIR) / "cyclobutane.qdxf.json"
with open(cyclobutane_qdxf_input_file, "w") as f:
    json.dump(cyclobutane_qdxf, f)

## 2.1) Arguments

In [ ]:
params = {
    "use_internal_coords": True,
    "max_iterations": 100,
    "conv_threshold": 1e-4,
    "basis_set": "cc-pVDZ",
}

## 2.2) Call geometry optimisation over the input geometry

In [ ]:
QM_GEO_OPT_RESOURCES = {
    "gpus": 1,
    "storage": 1024_000,
    "walltime": 60,
}
(geo_opt_out, results) = await client.qm_geo_opt(
    cyclobutane_qdxf_input_file, params, resources=QM_GEO_OPT_RESOURCES
)
print(await geo_opt_out.get())

## 2.3) Check that calculations converged

In [ ]:
results_out = await results.get()
assert results_out["converged"]

# 3) Visualise pre-optimised and optimised geometries

In [ ]:
import py3Dmol

In [ ]:
pre_opt_view = py3Dmol.view()
pre_opt_view.addModel(cyclobutane_xyz, "xyz")
pre_opt_view.setStyle({"stick": {}})
pre_opt_view.zoomTo()
pre_opt_view.show()

In [ ]:
topologies_out = await geo_opt_out.get()
opt_cyclobutane_xyz = qdxf_topology_to_xyz(
    topologies_out[-1], "optimised cyclobutane"
)

opt_view = py3Dmol.view()
opt_view.addModel(opt_cyclobutane_xyz, "xyz", {})
opt_view.setStyle({"stick": {}})
opt_view.zoomTo()
opt_view.show()